In [2]:
import numpy as np
import pandas as pd
from numpy.core.defchararray import add
import os


# refine subthreshold lensed injections set

In [16]:
data= dict(np.load('lensed_data/sub_sub_lensed.npz')) 

In [17]:
data.keys()

dict_keys(['dist', 't0', 'tensor_snr', 'idx_of_pkl', 'm1', 'm2', 'ra', 'dec', 'incl', 'pol', 'phi0', 'dl_un'])

In [18]:
for key in data.keys():
    data[key]=data[key].astype('float64')

In [19]:
ntot=data['m1'].shape[0]
data['event_tag'] = add('sub_sub_',np.arange(ntot).astype(str)).astype('U256')
data['img_tag'] = np.vstack([np.zeros(ntot),np.ones(ntot)]).T.astype(int)

In [21]:
np.savez('lensed_data/sub_sub_lensed_refined.npz',**data)

# refine subthreshold unlensed injections set

In [2]:
data = pd.read_csv("unlensed_data/unlensed_sub_Dominik_powerlaw2_inj_samples_withsnr.dat",delimiter='\t')

ntot=data['m1z'].shape[0]

data['phi0']=2*np.pi*np.random.rand(ntot)

In [3]:
data_dict = {}


data_dict['event_tag'] = add('sub_',np.arange(ntot).astype(str))
for key in data.columns:
    data_dict[key] = data[key].values.astype('float64')


In [4]:
data_dict['event_tag']= data_dict['event_tag'].astype(str)

np.savez('unlensed_data/sub_unlensed_refined.npz',**data_dict)

# create unlensed super sub pairs dataframe

In [52]:
#sup_unlensed_data = np.load('../data/injection_pars/training/analytical_psd_Dominik_powerlaw2_inj_samples_withsnr_refined.npz')
sup_unlensed_data = np.load('../data/injection_pars/haris-et-al/unlensed_inj_data.npz')

In [53]:
sub_unlensed_data = np.load('unlensed_data/sub_unlensed_refined.npz')

In [54]:
sup_unlensed_data['m1z'].shape

(1000,)

In [55]:
sub_unlensed_data['m1z'].shape

(1187,)

In [56]:
import os
odir = '/home/srashti.goyal/lensid_runs/subthreshold/ML_1p0_AnalyticalPsd_sub_unlensed/data/dataframes'
outfile = 'unlensed_super_sub_test.csv'
cols = ['img_0', 'img_1', 'Lensing']
img1,img2 = np.meshgrid(sub_unlensed_data['event_tag'],sup_unlensed_data['event_tag'][:1000])
img1=img1.flatten()
img2=img2.flatten()
npairs = img1.shape[0]
Unlensed_df_test = pd.DataFrame(columns=cols, index=range(npairs))
Unlensed_df_test['img_0'] = img1.astype('U256')
Unlensed_df_test['img_1'] = img2.astype('U256')
Unlensed_df_test['Lensing'] = np.zeros(npairs).astype('int')
print(npairs, 'no. of pairs in total ')
if not os.path.exists(odir):
    os.makedirs(odir)
outfile = odir + '/' + outfile
Unlensed_df_test.to_csv(outfile)
print('Dataframe saved at ', outfile)



1187000 no. of pairs in total 
Dataframe saved at  /home/srashti.goyal/lensid_runs/subthreshold/ML_1p0_AnalyticalPsd_sub_unlensed/data/dataframes/unlensed_super_sub_test.csv


# relative path for super sub unlensed pairs

In [44]:
import os
data_dir_sub_unlensed_qts = '/home/srashti.goyal/lensid_runs/subthreshold/ML_1p0_AnalyticalPsd_sub_unlensed/data/qts/'

In [45]:
data_dir_unlensed_super_qts = '/home/srashti.goyal/lensid_runs/ML_1p0_AnalyticalPsd/data/qts/'

In [46]:
rel_path = os.path.relpath(data_dir_sub_unlensed_qts, data_dir_unlensed_super_qts)

In [47]:
data_dir_unlensed_super_qts+rel_path

'/home/srashti.goyal/lensid_runs/ML_1p0_AnalyticalPsd/data/qts/../../../subthreshold/ML_1p0_AnalyticalPsd_sub_unlensed/data/qts'

# Copy superthreshold files to subthreshold unlensed folder

In [58]:
#unlensed_sup_sub_df = pd.read_csv('/home/srashti.goyal/lensid_runs/subthreshold/ML_1p0_AnalyticalPsd_sub_unlensed/data/dataframes/unlensed_super_sub.csv',index_col=0)
unlensed_sup_sub_df = pd.read_csv('/home/srashti.goyal/lensid_runs/subthreshold/ML_1p0_AnalyticalPsd_sub_unlensed/data/dataframes/unlensed_super_sub_test.csv',index_col=0)

In [59]:
unlensed_sup_sub_df.head()

,img_0,img_1,Lensing
0,sub_0,645858000.000000000-291716,0
1,sub_1,645858000.000000000-291716,0
2,sub_2,645858000.000000000-291716,0
3,sub_3,645858000.000000000-291716,0
4,sub_4,645858000.000000000-291716,0


In [60]:
super_tags=np.unique(unlensed_sup_sub_df['img_1'])

In [61]:
subthreshold_path = '/home/srashti.goyal/lensid_runs/subthreshold/ML_1p0_AnalyticalPsd_sub_unlensed/data/'

In [62]:
superthreshold_path = '/home/srashti.goyal/lensid_runs/ML_2p0_AnalyticalPsd/data/'

In [63]:
# copy bayestar skymaps
superthreshold_paths_sky=add(add(superthreshold_path+'bayestar_skymaps/test/',super_tags.astype(str)),'.npz')

In [64]:
import shutil

In [ ]:
[shutil.copy(file,subthreshold_path+'bayestar_skymaps/') for file in superthreshold_paths_sky]

In [66]:
# copy qts 
superthreshold_path_qts = '/home/srashti.goyal/lensid_runs/ML_1p0_AnalyticalPsd/data/'
superthreshold_paths_qts_H1=add(add(superthreshold_path_qts+'qts/test/H1/',super_tags.astype(str)),'.png')

In [ ]:
superthreshold_paths_qts_H1

In [ ]:
[shutil.copy(file,subthreshold_path+'qts/H1/') for file in superthreshold_paths_qts_H1]

In [69]:
superthreshold_paths_qts_L1=add(add(superthreshold_path_qts+'qts/test/L1/',super_tags.astype(str)),'.png')
superthreshold_paths_qts_V1=add(add(superthreshold_path_qts+'qts/test/V1/',super_tags.astype(str)),'.png')

In [ ]:
superthreshold_paths_qts_V1

In [ ]:
[shutil.copy(file,subthreshold_path+'qts/L1/') for file in superthreshold_paths_qts_L1]

In [ ]:
[shutil.copy(file,subthreshold_path+'qts/V1/') for file in superthreshold_paths_qts_V1]